<a href="https://colab.research.google.com/github/gracelcai/career-launch-group-13/blob/main/ExerciseModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import PyTorch
import torch
from torch import nn

# Import torchvision
import torchvision
from torchvision import datasets
from torchvision import transforms
from torchvision import transforms
from torchvision.transforms import ToTensor

# Import matplotlib
import matplotlib.pyplot as plt

# Check versions
print(torch.__version__)
print(torchvision.__version__)

In [ ]:
# Setup training data
train_data =

test_data =

In [ ]:
# Turn train dataset into DataLoader
from torch.utils.data import DataLoader

# setup batch size hyperparameter
BATCH_SIZE = 32
train_dataloader = DataLoader(dataset = train_data,
                           batch_size = BATCH_SIZE,
                           shuffle = True)

test_dataloader = DataLoader(dataset = test_data,
                             batch_size = BATCH_SIZE,
                             shuffle = False)

train_dataloader, test_dataloader